<h1 style="display:inline-block">ActiveVoxel Design</h1><br/>

subdivide( get_faces( solid ), divisions )

In [1]:
# cube_vxs = [[ rx ry rz invp id demi? ] ...]

# invp is the level of division of the unitary space
#  (the inverse of the line parameter in parametric form l = point.p + c)
# r in rx, ry, rz is a multiplier 0 <= r <= invp

# we use this form to keep integer coordinates, to prevent problems with precision
# and prevent ranges of tolerance to use the equality operator 

In [2]:
import numpy as np

vxs_py = [[ 1, 0, 0, 1, 0, 0 ], [ 0, 0, 0, 1, 1, 0 ], [ 0, 0, 1, 1, 2, 0 ], [ 1, 0, 1, 1, 3, 0 ], 
          [ 0, 1, 1, 1, 4, 0 ], [ 1, 1, 1, 1, 5, 0 ], [ 0, 1, 0, 1, 6, 0 ], [ 1, 1, 0, 1, 7, 0 ]]

vxs = np.array( vxs_py )

print( vxs.transpose())


[[1 0 0 1 0 1 0 1]
 [0 0 0 0 1 1 1 1]
 [0 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1]
 [0 1 2 3 4 5 6 7]
 [0 0 0 0 0 0 0 0]]


In [3]:
# TODO: useful?

# deriving vxs unique ids from their positions
# pmax is the max( invp ) 
pmax = 1

vx_names = {( pmax + 1 ) * ( pmax + 1 ) * vx[ 0 ] + ( pmax + 1 ) * vx[ 1 ] + vx[ 2 ]: 
            ( vx[ 3 ], vx[ 0 ], vx[ 1 ], vx[ 2 ], pmax ) for vx in vxs_py }

print( vx_names )


{4: (1, 1, 0, 0, 1), 0: (1, 0, 0, 0, 1), 1: (1, 0, 0, 1, 1), 5: (1, 1, 0, 1, 1), 3: (1, 0, 1, 1, 1), 7: (1, 1, 1, 1, 1), 2: (1, 0, 1, 0, 1), 6: (1, 1, 1, 0, 1)}


In [4]:
# edges divided in metadata and data to allow us to perform calculations with the smallest tensor possible

# vx1 vx2 id parent demi?

# we keep subdivided edges in an hierarchy. Edges can only be subdivided if we subdivide the faces they define.
# Two connected faces always share an edge. If we subdiv one of this faces, but not the other, the child edges 
# will be considered demi-edges (they exist in only one face).

# The same happens if we subdiv a face in n slices and the other face in m slices, if m is not a multiple of n.
# both sets of child edges will be kept and will be demi-edges, each set belonging to just one face.

# If we divide a face in n slices and the other face in n or kn (a multiple of n), the edges will be promoted
# to real edges. Their parent structures can be deleted or demoted to demi-geometries and kept for selection
# purposes (they can never be promoted to real geometry again). The default behavior is to delete.

edges_meta_py = [[ 0, 1, 0, 0, 0 ], [ 1, 2, 1, 0,  0 ], [ 2, 3, 2, 0,  0 ], [ 3, 0, 3, 0, 0 ],  
                 [ 2, 4, 4, 0, 0 ], [ 4, 5, 5,  0, 0 ], [ 5, 3, 6, 0,  0 ], [ 4, 6, 7, 0, 0 ], 
                 [ 6, 7, 8, 0, 0 ], [ 7, 5, 9, 0, 0 ], [ 6, 1, 10, 0, 0 ], [ 0, 7, 11, 0, 0 ]]


edges_meta = np.array( edges_meta_py )

max_edge = np.max( edges_meta, 0 )[ 2 ]

print( max_edge )

print( edges_meta.transpose())


11
[[ 0  1  2  3  2  4  5  4  6  7  6  0]
 [ 1  2  3  0  4  5  3  6  7  5  1  7]
 [ 0  1  2  3  4  5  6  7  8  9 10 11]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]]


In [5]:
# dx dy dz divs1 chdiv1 div2 chdiv2 id
# ( dx, dy, dz ) = direction vector = vx2 - vx1
# divs[n] = how many times this edge is divided in relation to face n
# chdiv[n] = children related to face n were created dividing this edge chdiv[n] times

# we keep v2 - v1 in the edges to allow us to make calculations to the edges directly without 
# use the vertices (this implies that if we change the vertices edges must be updated).

# in this data structure we keep the divisions of children edges, demi or real

edt_py = [[ vxs_py[ id[ 0 ]][ 0 ], 
            vxs_py[ id[ 0 ]][ 1 ], 
            vxs_py[ id[ 0 ]][ 2 ], 1, 0, 1, 0, id[ 2 ]] 
              for id in edges_meta_py ]

edges_data = np.array( edt_py )

edt_py2 = [[ vxs_py[ id[ 1 ]][ 0 ], 
            vxs_py[ id[ 1 ]][ 1 ], 
            vxs_py[ id[ 1 ]][ 2 ], 0, 0, 0, 0, 0 ] 
              for id in edges_meta_py ]

ed2 = np.array( edt_py2 )

edges_data -= ed2
edt_py = edges_data.tolist()

print( edt_py )
print()
print( edges_data.transpose())


[[1, 0, 0, 1, 0, 1, 0, 0], [0, 0, -1, 1, 0, 1, 0, 1], [-1, 0, 0, 1, 0, 1, 0, 2], [0, 0, 1, 1, 0, 1, 0, 3], [0, -1, 0, 1, 0, 1, 0, 4], [-1, 0, 0, 1, 0, 1, 0, 5], [0, 1, 0, 1, 0, 1, 0, 6], [0, 0, 1, 1, 0, 1, 0, 7], [-1, 0, 0, 1, 0, 1, 0, 8], [0, 0, -1, 1, 0, 1, 0, 9], [0, 1, 0, 1, 0, 1, 0, 10], [0, -1, 0, 1, 0, 1, 0, 11]]

[[ 1  0 -1  0  0 -1  0  0 -1  0  0  0]
 [ 0  0  0  0 -1  0  1  0  0  0  1 -1]
 [ 0 -1  0  1  0  0  0  1  0 -1  0  0]
 [ 1  1  1  1  1  1  1  1  1  1  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  1  1  1  1  1  1  1  1  1  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  2  3  4  5  6  7  8  9 10 11]]


In [6]:
# edge1 edge2 rev1 rev2 id demi? parent
# id < 0 = reversed edge

# two edges will be used to define tris. Just like edges tris, after subdivs,
# will be deleted and replaced by their children by default, but they can be kept as demi-tris for
# selection purposes

# rev[n] indicates if edge[n] should be considered reversed (2o vx -> 1o vx instead of 1o -> 2o)

tris_py = [[ 0,  1, 0, 0, 0, 0, 0 ], [  2,  3, 0, 0,  1, 0, 0 ], [  2, 4, 1, 0,  2, 0, 0 ], 
           [ 5,  6, 0, 0, 3, 0, 0 ], [  5,  7, 1, 0,  4, 0, 0 ], [  8, 9, 0, 0,  5, 0, 0 ], 
           [ 8, 10, 1, 0, 6, 0, 0 ], [  0, 11, 1, 0,  7, 0, 0 ], [ 11, 3, 1, 1,  8, 0, 0 ], 
           [ 6, 9 , 1, 1, 9, 0, 0 ], [ 10,  7, 1, 1, 10, 0, 0 ], [  4, 1, 1, 1, 11, 0, 0 ]]

tris = np.array( tris_py )
max_tri = np.max( tris, 0 )[ 2 ]

print( tris.transpose())


[[ 0  2  2  5  5  8  8  0 11  6 10  4]
 [ 1  3  4  6  7  9 10 11  3  9  7  1]
 [ 0  0  1  0  1  0  1  1  1  1  1  1]
 [ 0  0  0  0  0  0  0  0  1  1  1  1]
 [ 0  1  2  3  4  5  6  7  8  9 10 11]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]]


In [1]:
# subdiv tris algorithm

# (vxs algorithms are all abstracted in the promote and demote algorithms here)

# subdiv in n parts (n = 2 will result in n^2 = 4 tris )

# based on the tris selected (remove all demi-tris) we locate all edges 
# from all edges we select two groups: 
	# 1 - the ones that do not have n demi-edges as children
	# 2 - the ones that do have n demi-edges as children

# all in group 1 should be divided in n demi-edges, using # subdiv edges algorithm
# all children demi-edges in 2 should be promoted to real edges, using #promote to real edges

# select again all edges children of the original one, in two groups, one for each original edge
	# of each tri (so all the children of the first original edge of all tris in a group, and 
	# all children of the second original edge off all tris in the other )

	# the first group will be called "original row" and the second "original column"

# generate n - 1 rows of real edges corresponding to the tri subdiv, based on the original row, and add original row to it
# generate n - 1 cols of real edges corresponding to the tri subdiv, based on the original col, and add original col to it

# generate a cross of all rows x all cols based on shared vxs and angle of 90o degrees following
	# the space base (clockwise)
# add a cross of r x c based on shared vxs when edges point to the shared vx (invert cols)
# add a cross of r x c based on shared vxs when edges point away from it (invert rows)
# add a cross of r x c based on shared vxs based on angle of 90o degrees against the space base
	# (counter clockwise, invert rows and cols )

# generate tris 

# if the behaviour is preserve parent, demote old tris to demi-tris and link new tris,
	# using # demote tris
# or else delete old tris

# select all original edges that have real children and no demi-children
# demote all them to demi-edges, using # demote edges


# TODO: criar links para manter infos de parent em tris
# TODO: criar estrutura vx -> edge pra poder fazer o cross edges row x edges col

def subdivide( _faces, divs ):
    global max_tri, max_face, faces
    
    nnewf = divs * divs
    nnewt = nnewf * 2
    
    newf_py = [ id for id in range( max_tri + 1, max_tri + nnewt + 1 )]
    newf = np.array( newf_py )
    newf = np.reshape( newf, ( nnewf, 2 ))
    max_tri = max_tri + nnewt

    ids_py = [ id for id in range( max_face + 1, max_face + nnewf + 1 )]
    ids = np.array( ids_py )
    
    newf = np.c_[ newf, ids ]
    max_face = max_face + nnewf

    faces = np.vstack(( faces, newf ))
    
    _tris = [[ faces[ tri ][ 0 ], faces[ tri ][ 1 ]]  for tri in _faces ]
    _tris = [ item for sublist in _tris for item in sublist ]
    
    # TODO: pra criar os tris novos, tem q determinar as edges deles. Como as edges
    # ja podem existir, possivelmente é melhor continuar o algoritmo e gerar as 
    # novas edges primeiro
    
    _edges = [[ tris[ edge ][ 0 ], tris[ edge ][ 1 ]] for edge in _tris ]
    _edges = [ item for sublist in _edges for item in sublist ]
    print( _edges )

print( faces[ 0 ][ 0:1 ])
print()
subdivide( faces[ 0 ][ 0:1 ], 3 )


NameError: name 'faces' is not defined

In [ ]:
# translate to the viewer technology

# we will use vpython here, but the algorithm is probably the same or similar for all technologies

# generate vertex data
# generate tris data
# generate vertex normal data
# generate object data
# pass object to be rendered

from vpython import *

     
# generate vertex data - use TF to generate [ rx * 1/invp, ry * 1/invp, rz * 1/invp ] from vx tensor
#  first sort the vx tensor since the order of the index will change during operations
vxs = vxs[ vxs[ :, 4 ].argsort()]

vdata = 1.0 / vxs[:, 3 ]
vdata = np.multiply( vxs[:, [0, 1, 2]], vdata[:, None])

#print( vdata )

# generate tris data
# use edges do index vxs
    # TODO: edges precisa ser ordenada? Vao existir operacoes q tiram as edges de ordem? provavelmente sim
    
# vxs of 1st edge
egdata = edges_meta[ tris[:, 0 ], 0:2 ]

    # reversed edges ( rev1 )
rev = tris[:, 2 ]
m = rev > 0
swap = egdata[ m, 0 ]

egdata[ m, 0 ] = egdata[ m, 1 ]
egdata[ m, 1 ] = swap

# vxs of 2nd edge
egdata2 = edges_meta[ tris[:, 1 ], 0:2 ]

    # reversed edges ( rev2 )
rev = tris[:, 3 ]
m = rev > 0
swap = egdata2[ m, 0 ]

egdata2[ m, 0 ] = egdata2[ m, 1 ]
egdata2[ m, 1 ] = swap

tdata = np.c_[ egdata, egdata2[ :, 1 ]]
# generate vxs in the view technology
# using vpython

vpvecs = [ vec( vdata[ k, 0 ], vdata[ k, 1 ], vdata[ k, 2 ]) for k in range( vdata.shape[ 0 ])]

vpvxs = [ vertex( pos=vpvecs[ k ], color=vec( float(( k & 4 ) / 4 ), float(( k & 2 ) / 2 ), float( k & 1 ))) 
             for k in range( vdata.shape[ 0 ])]

# generate tris
vptris = [ triangle( vs=[ vpvxs[ tdata[ k, 0 ]], vpvxs[ tdata[ k, 1 ]], vpvxs[ tdata[ k, 2 ]]]) 
                       for k in range( tdata.shape[ 0 ])]

# generate solid
vpobj = compound( vptris )
vpobj.visible = True

# generating edges ( just to visualize them in vpython, this step is not necessary in all viewing technologies)
vpedges1 = [ curve( pos=[ vpvecs[ tdata[ k, 0 ]], vpvecs[ tdata[ k, 1 ]]], 
                   color=vec( 0.5,0.5,0.5 )) for k in range( tdata.shape[ 0 ])]

vpedges2 = [ curve( pos=[ vpvecs[ tdata[ k, 1 ]], vpvecs[ tdata[ k, 2 ]]], 
                   color=vec( 0.5,0.5,0.5 )) for k in range( tdata.shape[ 0 ])]

vpedges3 = [ curve( pos=[ vpvecs[ tdata[ k, 2 ]], vpvecs[ tdata[ k, 0 ]]], 
                   color=vec( 0.5,0.5,0.5 )) for k in range( tdata.shape[ 0 ])]

# init vpython and show the solid
#scene = canvas() # for jupyter notebooks
#scene.forward = vec( 0, -1, 0 )

#c = curve(vector(-1,-1,0), vector(1,-1,0))

while True:
    rate( 60 )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
test = [ ( float(( k & 4 ) / 4 ), float(( k & 2 ) / 2 ), float( k & 1 )) for k in range( vdata.shape[ 0 ])]
print( test )

[(0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (0.0, 1.0, 0.0), (0.0, 1.0, 1.0), (1.0, 0.0, 0.0), (1.0, 0.0, 1.0), (1.0, 1.0, 0.0), (1.0, 1.0, 1.0)]
